In [1]:
# Author: Davide Aloi - PhD student - University of Birmingham
# Description: the script extracts DCM values (pre < post) for all three datasets, and 
# saves them in .npy (numpy) files. These values will then be used in the correlation analysis,
# to assess the relationship between current density metrics and effective connectivity changes.
# The script also saves the thresholded DCM matrices (i.e. keeping only values which associated
# posterior probability > 75%).
import warnings
warnings.filterwarnings("ignore")
import os
import scipy.io
import numpy as np

# Output folder
output_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\all_dcm_results\\'

In [2]:
# DCM results paths
path_wp1a = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\wp1a_DCMfiles\\' # not yet available

In [3]:
## WP2A DCM RESULTS EXTRACTION 

# Day1_all_EPvalues.mat contains 66 DCM matrices (anodal, cathodal and sham, per participant (N=22)).
# This are the results of the comparison Day-1 pre < Day-1 post, per subject, per stimulation condition (anodal, cathodal, sham)
# Connections in the matrix are (in this order): M1, Th, SMA, Cerebellum.
# The DCM quantifies the change in effective connectivity after stimulation as compared to pre stimulation (contrast: pre < post).
# NB: I am interested only in anodal (indexes 0, 3, 6 ... 63) and sham (indexes 2, 5, 8... 65)
# NB: Each structure in the Day1_all_EPvalues.mat file contains an array of size 16. To get the DCM matrix,
# each array has to be reshaped (4,4) and transposed, because the DCM "thinks" columns (from) by rows (to) (rather than 
# rows x columns like everyone else on this planet).
# NB: The Day_1_all_EPvalues.mat file was created with the script wp2a_roast_4_extract_single_dcms.m

# Path to DCM and posterior probability results
path_wp2a = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\wp2a_DCMfiles\\'

# DCM data (EP values)
dcm_path = os.path.join(path_wp2a, 'Day1_all_EPvalues.mat') 
dcm_data = scipy.io.loadmat(dcm_path)
# Posterior probability
pp_data_path = os.path.join(path_wp2a, 'Day1_all_Pp.mat') 
pp_data = scipy.io.loadmat(pp_data_path)

# Variables where results will be saved
all_dcm_matrices = [] # DCM matrices for the anodal condition
all_dcm_matrices_sham = [] # DCM matrices for the sham condition
all_dcm_matrices_pp = [] # DCM matrices for the anodal condition (thresholded)
all_dcm_matrices_sham_pp = [] # DCM matrices for the sham condition (thresholded)

s = 0 # DCM index
for i in range(0, 22):
    # DCM data for subject i (s = anodal, s+2 = sham)
    this_dcm = np.array(dcm_data['ALLEP'][0][s].todense().reshape((4,4))).T # Anodal
    this_dcm_sham = np.array(dcm_data['ALLEP'][0][s + 2].todense().reshape((4,4))).T  # Sham

    # PP data for subject i 
    this_pp = pp_data['BMAs'][0][s]['Pp'][0][0][16:32].reshape([4,4]).T
    this_pp_sham = pp_data['BMAs'][0][s + 2]['Pp'][0][0][16:32].reshape([4,4]).T 

    # Storing DCM matrices
    all_dcm_matrices.append(this_dcm)
    all_dcm_matrices_sham.append(this_dcm_sham)

    # Storing DCM matrices thresholded (pp > 0.75)
    all_dcm_matrices_pp.append(np.where(this_pp > 0.75, this_dcm, 0)) # Anodal
    all_dcm_matrices_sham_pp.append(np.where(this_pp_sham > 0.75, this_dcm_sham, 0)) # Sham

    s = s + 3 # moving to next subject

# Saving everything (unthresholded and thresholded DCM matrices for each subject)
# NB this saves np.arrays of size (22, 4, 4). To load use np.load(path)
np.save(os.path.join(output_folder, 'wp2a_dcm_unthresholded'), np.array(all_dcm_matrices))
np.save(os.path.join(output_folder, 'wp2a_dcm_sham_unthresholded'), np.array(all_dcm_matrices_sham))
np.save(os.path.join(output_folder, 'wp2a_dcm_thresholded'), np.array(all_dcm_matrices_pp))
np.save(os.path.join(output_folder, 'wp2a_dcm_sham_thresholded'), np.array(all_dcm_matrices_sham_pp))

In [4]:
## WP1B DCM RESULTS EXTRACTION

# For this dataset I'm interested in cathodal and sham. 
# The order in this dataset is sham, cathodal, anodal. The number of subjects is 21 (63 DCM
# matrices in total). 
# NB: the way we access values inside wp1b_all_EPvalues.mat and All_Pp_wp1b.mat is the same 
# as dataset wp2a (see above)

# Path to DCM and posterior probability results
path_wp1b = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\wp1b_DCMfiles\\'

# DCM data (EP values)
dcm_path_wp1b = os.path.join(path_wp1b, 'wp1b_all_EPvalues.mat') 
dcm_data_wp1b = scipy.io.loadmat(dcm_path_wp1b)
# Posterior probability
pp_data_path_wp1b = os.path.join(path_wp1b, 'All_Pp_wp1b.mat') 
pp_data_wp1b = scipy.io.loadmat(pp_data_path_wp1b)

# Variables where results will be saved
all_dcm_matrices_wp1b = [] # DCM matrices for the cathodal condition
all_dcm_matrices_sham_wp1b = [] # DCM matrices for the sham condition
all_dcm_matrices_pp_wp1b = [] # DCM matrices for the cathodal condition (thresholded)
all_dcm_matrices_sham_pp_wp1b = [] # DCM matrices for the sham condition (thresholded)

s = 0 # DCM index
for i in range(0, 21):
    # DCM data for subject i (s+1 = cathodal, s = sham)
    this_dcm = np.array(dcm_data_wp1b['ALLEP'][0][s + 1].todense().reshape((4,4))).T # Cathodal
    this_dcm_sham = np.array(dcm_data_wp1b['ALLEP'][0][s].todense().reshape((4,4))).T  # Sham

    # PP data for subject i 
    this_pp = pp_data_wp1b['BMAs'][0][s + 1]['Pp'][0][0][16:32].reshape([4,4]).T
    this_pp_sham = pp_data_wp1b['BMAs'][0][s]['Pp'][0][0][16:32].reshape([4,4]).T 

    # Storing DCM matrices
    all_dcm_matrices_wp1b.append(this_dcm)
    all_dcm_matrices_sham_wp1b.append(this_dcm_sham)

    # Storing DCM matrices thresholded (pp > 0.75)
    all_dcm_matrices_pp_wp1b.append(np.where(this_pp > 0.75, this_dcm, 0)) # Cathodal
    all_dcm_matrices_sham_pp_wp1b.append(np.where(this_pp_sham > 0.75, this_dcm_sham, 0)) # Sham

    s = s + 3 # moving to next subject


# Saving everything (unthresholded and thresholded DCM matrices for each subject)
# NB this saves np.arrays of size (21, 4, 4). To load use np.load(path)
np.save(os.path.join(output_folder, 'wp1b_dcm_unthresholded'), np.array(all_dcm_matrices_wp1b))
np.save(os.path.join(output_folder, 'wp1b_dcm_sham_unthresholded'), np.array(all_dcm_matrices_sham_wp1b))
np.save(os.path.join(output_folder, 'wp1b_dcm_thresholded'), np.array(all_dcm_matrices_pp_wp1b))
np.save(os.path.join(output_folder, 'wp1b_dcm_sham_thresholded'), np.array(all_dcm_matrices_sham_pp_wp1b))   

In [ ]:
## WP1A DCM RESULTS EXTRACTION

# Waiting for Davinia to send me the DCM files
